# Demo
Examples of how to use this repo

These are the commands for VPT with `p=50`:

VPT-deep
```bash
    --config-file configs/prompt/*.yaml
    MODEL.TRANSFER_TYPE "prompt" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.NUM_TOKENS "50" \
    MODEL.PROMPT.DROPOUT "0.0" 
```
   
VPT-shallow (we don't use dropout for VPT-shallow)
```bash
    --config-file configs/prompt/*.yaml
    MODEL.TRANSFER_TYPE "prompt" \
    MODEL.PROMPT.DEEP "False" \
    MODEL.PROMPT.NUM_TOKENS "50" \
    MODEL.PROMPT.DROPOUT "0.0" 
```

Other transfer protocols presented in the paper:

Full
```bash
    --config-file configs/finetune/*.yaml
```

Head-oriented methods:

- Linear:
```bash
    --config-file configs/linear/*.yaml
```

- MLP-3 (3 layer MLP):
```bash
    --config-file configs/linear/*.yaml \
    MODEL.MLP_NUM "2"
```

- Partial-1:
```bash
    --config-file configs/finetune/*.yaml \
    MODEL.TRANSFER_TYPE "partial-1"
```


Backbone-oriented methods:

- Sidetune:
```bash
    --config-file configs/linear/*.yaml
    MODEL.TRANSFER_TYPE  "side" 
```

- Bias: 
```bash
    --config-file configs/finetune/*.yaml \
    MODEL.TRANSFER_TYPE "tinytl-bias"
```

- Adapters with `r=128`:
```bash
    --config-file configs/finetune/*.yaml
    MODEL.ADAPTER.REDUCATION_FACTOR "128"
    MODEL.TRANSFER_TYPE "adapter" 
```

##  train.py
The main script is `train.py`. Note for VTAB data, this script handles the final runs with 1000 training data. See `tune_vtab.py` for the full tune + final runs settings. Here are some examples.

Note: it's recommended to directly use terminal for these command.

In [1]:
import torch

/home/yourenz/.conda/envs/prompt/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%%bash

source activate prompt

model_root=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/
data_path=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/datasets/cub
output_dir=/home/yourenz/SynergisticEfficiency/exps

python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitti16_imagenet21k" \
        SEED 42 \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}" 

In [ ]:
%%bash
# launch final training with five random seeds for VTAB-dmlab, sun397 and eurosat. The hyperparameters are the same from our paper.
model_root=<MODEL_ROOT>
data_path=<DATA_PATH>
output_dir=<OUTPUT_DIR>
        
# vtab-structured: dmlab
# base_lr = 1.0
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-dmlab" \
        DATA.NUMBER_CLASSES "6" \
        SOLVER.BASE_LR "0.25" \
        SOLVER.WEIGHT_DECAY "0.001" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

# vtab-natural: sun397
# base_lr = 25
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "128" \
        MODEL.PROMPT.NUM_TOKENS "5" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-sun397" \
        DATA.NUMBER_CLASSES "397" \
        SOLVER.BASE_LR "12.5" \
        SOLVER.WEIGHT_DECAY "0.0001" \
        SOLVER.TOTAL_EPOCH "100" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

# vtab-specialized: vtab-eurosat
# base_lr = 1
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-eurosat" \
        DATA.NUMBER_CLASSES "10" \
        SOLVER.BASE_LR "0.25" \
        SOLVER.WEIGHT_DECAY "0.001" \
        SOLVER.TOTAL_EPOCH "100" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

## Get results

In [1]:
import glob
import pandas as pd

from src.utils.vis_utils import get_df, average_df

LOG_NAME = "logs.txt"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/home/chuizhao/.conda/envs/prompt/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = <MODEL_ROOT>
df_list=[]
for seed in ["42", "44", "82", "100", "800"]:
#     model_type = f"adapter_{r}"
    files = glob.glob(f"{root}/seed{seed}/*/sup_vitb16_imagenet21k/*/*/{LOG_NAME}")
    for f in files:
        df = get_df(files, f"seed{seed}", root, is_best=False, is_last=True)
        if df is None:
            continue
        df["seed"] = seed
    df_list.append(df)

df= pd.concat(df_list)
df["type"] = "VPT"
df

seed800: 100%|██████████| 3/3 [00:00<00:00, 252.79it/s]


,data,feature,lr,wd,total_params,tuned_params,tuned / total (%),batch_size,l-val_top1,l-test_top1,best_epoch,file,total_time,seed,type
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,46.88,76 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:03,42,VPT
0,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.00,38 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:57,42,VPT
1,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,52.57,14 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:57,42,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,99.5,46.25,85 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:09,44,VPT
0,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.54,41 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:38,44,VPT
1,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,49.03,32 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:53,44,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,46.14,65 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:04:18,82,VPT
1,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.67,41 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:42,82,VPT
0,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,52.45,8 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:51,82,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,47.41,76 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:15,100,VPT


Take average of 5 runs for each dataset:

In [4]:
# LR represents the base learning rate, not the scaled one.
f_df = average_df(df, metric_names=["l-test_top1"], take_average=True)
f_df

,data,feature,type,total_runs,l-test_top1,l-test_top1-std,lr,wd,total_params,tuned_params,tuned / total (%),batch_size,l-val_top1,total_time,seed
0,vtab-dmlab,sup_vitb16_imagenet21k,VPT,5,46.62,0.47,1.0,0.0010,86724870,926214,1.0680,64,100.0,0 days 01:06:03,42
2,vtab-eurosat,sup_vitb16_imagenet21k,VPT,5,96.15,0.45,1.0,0.0010,86727946,929290,1.0715,64,100.0,0 days 00:44:57,42
1,vtab-sun397,sup_vitb16_imagenet21k,VPT,5,51.57,1.40,25.0,0.0001,86150029,351373,0.4079,128,100.0,0 days 00:44:57,42


## tune*.py
Tune vtab or fgvc datasets. 

In [ ]:
%%bash
# Tune VTAB-caltech101 with VPT:
python tune_vtab.py \
    --train-type "prompt" \
    --config-file configs/prompt/cub.yaml \
    MODEL.TYPE "vit" \
    DATA.BATCH_SIZE "128" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.DROPOUT "0.1" \
    MODEL.PROMPT.NUM_TOKENS "10" \
    DATA.FEATURE "sup_vitb16_imagenet21k" \
    DATA.NAME "vtab-caltech101" \
    DATA.NUMBER_CLASSES "102" \
    DATA.DATAPATH <DATA_PATH> \
    MODEL.MODEL_ROOT <MODEL_ROOT> \
    OUTPUT_DIR <OUTPUT_PATH> 

# Tune CUB with VPT:
python tune_fgvc.py \
    --train-type "prompt" \
    --config-file configs/prompt/cub.yaml \
    MODEL.TYPE "vit" \
    DATA.BATCH_SIZE "128" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.DROPOUT "0.1" \
    MODEL.PROMPT.NUM_TOKENS "10" \
    DATA.FEATURE "sup_vitb16_imagenet21k" \
    DATA.DATAPATH <DATA_PATH> \
    MODEL.MODEL_ROOT <MODEL_ROOT> \
    OUTPUT_DIR <OUTPUT_PATH> 

## Backbone choices

- Swin-B

```bash
    MODEL.TYPE "swin" \
    DATA.FEATURE "swinb_imagenet22k_224"
```

- ResNet-50 (VPT with prompt location == pad)

```bash
    MODEL.TYPE "resnet" \
    DATA.FEATURE "imagenet_sup_rn50" \
    SOLVER.OPTIMIZER "sgd" \
    MODEL.PROMPT.LOCATION "pad" \
    MODEL.PROMPT.NUM_TOKENS "5" 
```

- ConvNeXt-Base (VPT with prompt location == pad)

```bash
    MODEL.TYPE "resnext" \
    DATA.FEATURE "imagenet22k_sup_rnx_base" \
    MODEL.PROMPT.LOCATION "pad" \
    MODEL.PROMPT.NUM_TOKENS "5" 
```

ViT with self-supervised pre-training objectives:
    
- MAE

```bash
MODEL.TYPE "ssl-vit" \
DATA.FEATURE "mae_vitb16"
```

- MoCo-v3

```bash
MODEL.TYPE "ssl-vit" \
DATA.FEATURE "mocov3_vitb" 
```

## ToMe

In [4]:
from src.configs.config import get_cfg
from src.models.build_model import build_model
model_root='/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/'

In [5]:
cfg_file = './configs/prompt/cub.yaml'
cfg = get_cfg()
cfg.merge_from_file(cfg_file)
cfg.MODEL.MODEL_ROOT = model_root
cfg.DATA.FEATURE = 'sup_vitti16_imagenet21k'
cfg.NUM_GPUS = 0
cfg

CfgNode({'DBG': False, 'OUTPUT_DIR': '', 'RUN_N_TIMES': 1, 'CUDNN_BENCHMARK': False, 'NUM_GPUS': 0, 'NUM_SHARDS': 1, 'SEED': None, 'MODEL': CfgNode({'TRANSFER_TYPE': 'prompt', 'WEIGHT_PATH': '', 'SAVE_CKPT': False, 'REDUCTION': 13, 'MODEL_ROOT': '/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/', 'TYPE': 'vit', 'MLP_NUM': 0, 'LINEAR': CfgNode({'MLP_SIZES': [], 'DROPOUT': 0.1}), 'PROMPT': CfgNode({'NUM_TOKENS': 5, 'LOCATION': 'prepend', 'INITIATION': 'random', 'CLSEMB_FOLDER': '', 'CLSEMB_PATH': '', 'PROJECT': -1, 'DEEP': False, 'NUM_DEEP_LAYERS': None, 'REVERSE_DEEP': False, 'DEEP_SHARED': False, 'FORWARD_DEEP_NOEXPAND': False, 'VIT_POOL_TYPE': 'original', 'DROPOUT': 0.0, 'SAVE_FOR_EACH_EPOCH': False}), 'ADAPTER': CfgNode({'REDUCATION_FACTOR': 8, 'STYLE': 'Pfeiffer'})}), 'SOLVER': CfgNode({'LOSS': 'softmax', 'LOSS_ALPHA': 0.01, 'OPTIMIZER': 'sgd', 'MOMENTUM': 0.9, 'WEIGHT_DECAY': 0.01, 'WEIGHT_DECAY_BIAS': 0, 'PATIENCE': 300, 'SCHEDULE

In [8]:
import tome, timm

In [6]:
model_vpt, cur_device = build_model(cfg)
model_vpt.__class__

src.models.vit_models.ViT

In [7]:
import torch
input = torch.rand((4, 3, 224, 224))
output_vpt = model_vpt(input)
print(output_vpt.shape)

torch.Size([4, 200])


In [13]:
tome.patch.timm(model_vpt, trace_source=True)
model_vpt.r = 13
model_vpt.__class__

replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention
replace the block with tomeblock
replace the attention with tomeattention


tome.patch.timm.make_tome_class.<locals>.ToMeVisionTransformer

In [16]:
import torch
# tome.patch.timm(model_vpt, prop_attn=False)
model_vpt.r = 13
input = torch.rand((4, 3, 224, 224))
output_vpt = model_vpt(input)
print(output_vpt.shape)
print(model_vpt._tome_info["source"].shape)

torch.Size([4, 200])
torch.Size([4, 46, 202])


In [ ]:
import inspect
for i in inspect.getmembers(model_vpt):
     
    # to remove private and protected
    # functions
    if not i[0].startswith('_'):
         
        # To remove other methods that
        # doesnot start with a underscore
        if not inspect.ismethod(i[1]):
            print(i)

In [ ]:
for i in inspect.getmembers(model_timm):
     
    # to remove private and protected
    # functions
    if not i[0].startswith('_'):
         
        # To remove other methods that
        # doesnot start with a underscore
        if not inspect.ismethod(i[1]):
            print(i)

In [20]:
import torch
import torch.nn as nn
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
        # k: [b, num_heads, N, dim // num_heads]
        print(k.shape)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [25]:
input = torch.rand((4, 10, 72))
attn = Attention(dim=72)

In [26]:
ouput = attn(input)
print(ouput.shape)

torch.Size([4, 8, 10, 9])
torch.Size([4, 10, 72])


## Download Dataset

In [1]:
import tensorflow_datasets as tfds
data_dir = "/gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/datasets/"

2023-04-10 17:08:19.621553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 17:08:21.356479: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/pkgs/arc/cuda/11.3.0/lib64:/opt/slurm/lib64::
2023-04-10 17:08:21.356607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/pkgs/arc/cuda/11.3.0/lib64:/opt/slurm/lib64::
2023-04-10 17:08:21.35

In [3]:
dataset_builder = tfds.builder("caltech101:3.*.*", data_dir=data_dir)
dataset_builder.download_and_prepare()

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00,  3.18 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00,  3.13 url/s]


NonMatchingChecksumError: Artifact https://drive.google.com/uc?export=download&id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp, downloaded to /gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/datasets/downloads/ucexport_download_id_137RyRjvTBkBiIfeYBNZ_EwspXlUxYffGj8eiXhr_4rE3NXcywQLRalS5KkycsuJfYfM.tmp.2328f077a9e5415c9d2dc3cc8d6d86d7/uc, has wrong checksum:
* Expected: UrlInfo(size=125.64 MiB, checksum='af6ece2f339791ca20f855943d8b55dd60892c0a25105fcd631ee3d6430f9926', filename='view')
* Got: UrlInfo(size=2.24 KiB, checksum='206003c9c1f7fd8a1457c7c82f7943b73255e17a755c80c98514bbbe510a9115', filename='uc')
To debug, see: https://www.tensorflow.org/datasets/overview#fixing_nonmatchingchecksumerror

In [4]:
dataset_builder = tfds.builder("cifar100:3.*.*", data_dir = data_dir)
dataset_builder.download_and_prepare()

2023-03-27 20:24:27.591595: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:03<00:00,  3.46s/ url]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]2023-03-27 20:24:45.874276: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /sw/pkgs/arc/cuda/11.3.0/lib64:/opt/slurm/lib64::
2023-03-27 20:24:45.874312: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-27 20:24:45.874331: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl-login1.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2023-03-27 20:24:45.906217: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (one

Dataset cifar100 downloaded and prepared to /gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


In [5]:
subset="rgb"
dataset_name = "eurosat/{}:2.*.*".format(subset)
dataset_builder = tfds.builder(dataset_name, data_dir=data_dir)
dataset_builder.download_and_prepare()

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:05<00:00,  5.64s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:41<00:00, 41.47s/ url]
                                                                         

Dataset eurosat downloaded and prepared to /gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.


In [ ]:
dataset_builder = tfds.builder("sun397/tfds:4.*.*", data_dir=data_dir)
dataset_builder.download_and_prepare()

In [6]:
dataset_builder = tfds.builder("dmlab:2.0.1", data_dir=data_dir)
dataset_builder.download_and_prepare()

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [01:08<00:00, 68.23s/ url]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]2023-03-27 21:11:24.379223: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
                                                                         

Dataset dmlab downloaded and prepared to /gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/datasets/dmlab/2.0.1. Subsequent calls will reuse this data.
